# Transfer Learning with VGG 19

## Import Libraries

In [3]:
from keras.applications.vgg19 import VGG19
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Flatten
import matplotlib.pyplot as plt

## Data Preprocessing

In [ ]:
train_data_dir = 'Dataset/train'
validation_data_dir = 'Dataset/test'

### Preview Data

### Image Augmentation

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   
                                   )
validation_datagen = ImageDataGenerator(rescale=1./255)

## Modelling

### Pre-Model Load

In [ ]:
# Load the pre-trained VGG19 model
vgg19 = VGG19(weights='imagenet',
              include_top=False,
              input_shape=(224, 224, 3))

# Freeze the weights of the pre-trained layers
for layer in vgg19.layers:
    layer.trainable = False

# Add your custom layers on top of the pre-trained model
model = Sequential([
    vgg19,
    Flatten(),
    Dense(256, activation='relu'),
    Dense(10, activation='softmax'),
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

### Fine Tuning

In [ ]:
# Set up the data generator for training and validation
train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size=(224, 224),
                                                    batch_size=32,
                                                    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(validation_data_dir,
                                                              target_size=(224, 224),
                                                              batch_size=32,
                                                              class_mode='categorical')

# Fine-tune the model on your dataset
history = model.fit_generator(train_generator,
                              steps_per_epoch=2000 // 32, # number of batches per epoch
                              epochs=10,
                              validation_data=validation_generator,
                              validation_steps=800 // 32) # number of validation batches

## Model Evaluation

### Accuracy

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

### Loss

In [ ]:
plt.subplot(2, 1, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

## Save the Model

In [ ]:
# Save the model
# model.save('e_waste_classifier.h5')